# Reproduction of  `epsilon` metric

If you wish to reproduce the results presented in our paper from scratch, feel free to use the below code.
In this notebook, we provide the codes to reproduce the results for NAS-Bench-101 sarch space, CIFAR10 dataset.

In [1]:
import os
import json
import time
import itertools

import numpy as np
import pandas as pd
import pickle as pkl
import seaborn as sns
from scipy import stats
from tqdm import trange
from dotmap import DotMap
from statistics import mean
import matplotlib.pyplot as plt
from prettytable import PrettyTable

import torch.nn as nn
import torch

import nasspace
from datasets import data
from epsilon_utils import prepare_seed, compute_epsilon

2023-03-15 09:17:08.448522: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64:/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64
2023-03-15 09:17:08.448643: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64:/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64
2023-03-15 09:17:08.448655: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above

In [2]:
dataset = 'cifar10'
data_loc = './datasets/cifardata'
batch_size = 256
repeat = 1
GPU = '0'
augtype = 'none'
trainval = False

In [3]:
# Arguments required for NAS-Bench-201 search space initialisation
args = DotMap()

args.api_loc = './nasbench_only108.tfrecord'
args.nasspace = 'nasbench101'
args.dataset = dataset
args.stem_out_channels = 128
args.num_stacks = 3
args.num_modules_per_stack = 3
args.num_labels = 1

savedataset = dataset
dataset = 'fake' if 'fake' in savedataset else savedataset
savedataset = savedataset.replace('fake', '')
if savedataset == 'cifar10':
    savedataset = savedataset + '-valid'

In [4]:
# Load the search space (it takes some time)
searchspace = nasspace.get_search_space(args)

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 73 seconds


In [5]:
if 'valid' in savedataset:
    savedataset = savedataset.replace('-valid', '')
    
if args.dataset == 'cifar10':
    acc_type = 'ori-test'
    val_acc_type = 'x-valid'
else:
    acc_type = 'x-test'
    val_acc_type = 'x-valid'

In [6]:
# Define the device
os.environ['CUDA_VISIBLE_DEVICES'] = GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Randomly select n_samples architectures
prepare_seed(21)

In [7]:
# Load the data 
train_loader = data.get_data(dataset, data_loc, trainval, batch_size, augtype, repeat, args)

# Pick up a batch
data_iterator = iter(train_loader)
x, _= next(data_iterator)
x = x.to(device)

Files already downloaded and verified
Files already downloaded and verified


In [18]:
save_dir = './release_results_ICML_11/{}/'.format(dataset.upper(), batch_size)
os.makedirs(save_dir, exist_ok=True)

weights = [-1, 1]
test_accs_mean = []
test_accs_min = []
test_accs_max = []
val_accs_mean = []
val_accs_min = []
val_accs_max = []
nparams = []
scores = []
times = []

for i in trange(len(searchspace)):
    start = time.time()
    uid = searchspace[i]
    network = searchspace.get_network(uid)
    network = network.to(device)
    score = compute_epsilon(x, network, weights)
    scores.append(score)
    nparams.append(sum(p.numel() for p in network.parameters()))
    test_accs_mean.append(searchspace.get_final_accuracy(uid, acc_type, False)[0])
    test_accs_min.append(searchspace.get_final_accuracy(uid, acc_type, False)[1])
    test_accs_max.append(searchspace.get_final_accuracy(uid, acc_type, False)[2])
    if dataset=='cifar10':
        val_accs_mean.append(searchspace.get_final_accuracy(uid, val_acc_type, True)[0])
        val_accs_min.append(searchspace.get_final_accuracy(uid, val_acc_type, True)[1])
        val_accs_max.append(searchspace.get_final_accuracy(uid, val_acc_type, True)[2])
    else:
        val_accs_mean.append(searchspace.get_final_accuracy(uid, val_acc_type, False)[0])
        val_accs_min.append(searchspace.get_final_accuracy(uid, val_acc_type, False)[1])
        val_accs_max.append(searchspace.get_final_accuracy(uid, val_acc_type, False)[2])
    times.append(time.time()-start)

# Save your results
save_dic = {} 
save_dic["scores"] = scores
save_dic["test_accs_mean"] = test_accs_mean
save_dic["test_accs_min"] =  test_accs_min
save_dic["test_accs_max"] =  test_accs_max
save_dic["val_accs_mean"] = val_accs_mean
save_dic["val_accs_min"] =  val_accs_min
save_dic["val_accs_max"] =  val_accs_max
save_dic["nparams"] = nparams
save_dic["times"] = times

pkl.dump(save_dic, open(save_dir + "Data", "wb"))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 423624/423624 [33:58:14<00:00,  3.46it/s]


In [10]:
save_dir

'./release_results/NAS-Bench-101/CIFAR10/INDIVIDUAL_WEIGHTS_1e-7_1/BS_256/NoInf/BatchMean/'

In [9]:
# Save your results
save_dir = './release_results/NAS-Bench-101/CIFAR10/WEIGHT_{}_{}/BS_{}/'.format(weight_l, weight_h, args.batch_size)
os.makedirs(save_dir, exist_ok=True)

save_dic = {}
save_dic["score"] = score
save_dic["accs_mean"] = accs_mean
save_dic["accs_min"] = accs_min
save_dic["accs_max"] = accs_max
save_dic["nparams"] = nparams
    
pkl.dump(save_dic, open(save_dir + "Data_1", "wb"))